Relaunched Hedge Funds
==

This program is a replication of Stata Script Version 0.2

In [1]:
import pandas as pd

# ****************** Program Settings ******************
Folder 		= "~/Dropbox/Projects/2015 - Lippers Tass Relaunched Hedge Funds/"		 # Location of program scripts
Data 		= "~/Workspace/Temp/"								 # // Location to which temporary files are generated
DataSource  = "~/Dropbox/Resources/Datasets/Lippers Tass/Tass_Access_13/"		 # // Location of the original data files (ASCII)
# ******************************************************

# Prepare Data

In [2]:
# Load manager details
data_manager = pd.read_stata(DataSource + 'PeopleDetails.dta')
data_manager = data_manager[data_manager.PersonTypeID==1]
variables_to_keep = "ProductReference PersonID First Last JobTitle Address1 Address2 Address3 CityName StateName Zip CountryName".split()
data_manager = data_manager[variables_to_keep]
data_manager.head()

,ProductReference,PersonID,First,Last,JobTitle,Address1,Address2,Address3,CityName,StateName,Zip,CountryName
1,5,2508,Robert,Thompson,Portfolio Manager,3351 W Bearss Ave,,,Tampa,Florida,33618,United States
4,8,139,Enrico,Massignani,Performance Contact,Via Ugo Bassi 6,,,Milan,None,20159,Italy
6,9,139,Enrico,Massignani,Performance Contact,Via Ugo Bassi 6,,,Milan,None,20159,Italy
9,10,139,Enrico,Massignani,Performance Contact,Via Ugo Bassi 6,,,Milan,None,20159,Italy
11,11,139,Enrico,Massignani,Performance Contact,Via Ugo Bassi 6,,,Milan,None,20159,Italy


In [3]:
# Extract Inception / PerformnaceEndData date
data_dates = pd.read_stata(DataSource + 'ProductDetails.dta')
variables_to_keep = "ProductReference InceptionDate PerformanceEndDate".split()
data_dates = data_dates[variables_to_keep]
data_dates.head()

,ProductReference,InceptionDate,PerformanceEndDate
0,86,1993-01-01,2013-09-30
1,87,1992-01-01,2008-09-30
2,90,1993-07-01,1997-12-31
3,91,1993-07-01,1997-12-31
4,92,1993-07-01,2000-12-31


# Analysis



Merge managers with the start/end dates

In [38]:
# Inner join - non-matches are excluded
data_merged = data_manager.merge(data_dates, on='ProductReference', how='inner')
data_merged = data_merged.sort(columns=['PersonID', 
                                        'PerformanceEndDate', 
                                        'InceptionDate',
                                        'ProductReference'])
data_merged.head()

,ProductReference,PersonID,First,Last,JobTitle,Address1,Address2,Address3,CityName,StateName,Zip,CountryName,InceptionDate,PerformanceEndDate
1197,1763,4,Douglas,Bry,President,3609 South Wadsworth Blvd.,Suite 250,,Denver,Colorado,80235,United States,1996-08-01,1998-03-31
312,637,4,Douglas,Bry,President,3609 South Wadsworth Blvd.,Suite 250,,Denver,Colorado,80235,United States,1991-03-01,2011-03-31
1120,1679,4,Douglas,Bry,President,3609 South Wadsworth Blvd.,Suite 250,,Denver,Colorado,80235,United States,1991-03-01,2011-03-31
1856,2508,9,Jerald,Weintraub,(none),900 Montgomery Street,,,San Francisco,California,94133,United States,1992-05-08,2006-10-31
2314,3721,9,Jerald,Weintraub,(none),900 Montgomery Street,,,San Francisco,California,94133,United States,1998-11-18,2007-05-31


Find **First End Date** for each PersonID.

In [61]:
# Create a GroupBy object
grouped = data_merged[['PersonID', 'PerformanceEndDate']].groupby('PersonID', axis=0)  # these will aplit the DataFrame on its index (rows).
grouped.groups
print(grouped.get_group(4))
grouped.min()

# Find the smallest value in the end date for each PersonID
transformed = grouped.min()
transformed['first_end_date'] = transformed['PerformanceEndDate']
transformed = transformed.drop('PerformanceEndDate', axis=1)

# Merge back
print(data_merged.PersonID.head())
print(type(transformed))
print(type(data_merged))
data_merged = data_merged.merge(transformed, how='outer', 
                                left_on='PersonID', right_index=True)
data_merged = data_merged.rename(columns={
    'first_end_date_x': 'first_end_date'
    }).drop('first_end_date_y', axis=1)
data_merged.head()

      PersonID PerformanceEndDate
1197         4         1998-03-31
312          4         2011-03-31
1120         4         2011-03-31
1197    4
312     4
1120    4
1856    9
2314    9
Name: PersonID, dtype: int32
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


,ProductReference,PersonID,First,Last,JobTitle,Address1,Address2,Address3,CityName,StateName,Zip,CountryName,InceptionDate,PerformanceEndDate,first_end_date,first_end_date,first_end_date,first_end_date
1197,1763,4,Douglas,Bry,President,3609 South Wadsworth Blvd.,Suite 250,,Denver,Colorado,80235,United States,1996-08-01,1998-03-31,1998-03-31,1998-03-31,1998-03-31,1998-03-31
312,637,4,Douglas,Bry,President,3609 South Wadsworth Blvd.,Suite 250,,Denver,Colorado,80235,United States,1991-03-01,2011-03-31,1998-03-31,1998-03-31,1998-03-31,1998-03-31
1120,1679,4,Douglas,Bry,President,3609 South Wadsworth Blvd.,Suite 250,,Denver,Colorado,80235,United States,1991-03-01,2011-03-31,1998-03-31,1998-03-31,1998-03-31,1998-03-31
1856,2508,9,Jerald,Weintraub,(none),900 Montgomery Street,,,San Francisco,California,94133,United States,1992-05-08,2006-10-31,2006-10-31,2006-10-31,2006-10-31,2006-10-31
2314,3721,9,Jerald,Weintraub,(none),900 Montgomery Street,,,San Francisco,California,94133,United States,1998-11-18,2007-05-31,2006-10-31,2006-10-31,2006-10-31,2006-10-31


,PerformanceEndDate
PersonID,
4,1998-03-31
9,2006-10-31
11,1997-04-30
15,1999-08-31
19,2002-03-31
22,2013-09-30
25,2004-11-30
26,2004-11-30
27,2004-11-30
